<a href="https://colab.research.google.com/github/ludoveltz/test_github_fev25/blob/main/Mini-projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation des packages nécessaires
!pip install -q langchain langchain_openai chromadb openai python-dotenv

# Imports
from google.colab import userdata
import os

try:
    # Configuration sécurisée de la clé API
    api_key = userdata.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("Clé API non trouvée dans les secrets Colab")

    os.environ['OPENAI_API_KEY'] = api_key
    print("✅ Clé API configurée avec succès")

    # Test rapide de la configuration
    from openai import OpenAI
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Test de connexion"}],
        max_tokens=5
    )
    print("✅ Test de connexion OpenAI réussi")

except Exception as e:
    print(f"❌ Erreur: {str(e)}")
    print("\nSi vous n'avez pas encore configuré votre clé API dans les secrets Colab:")
    print("1. Cliquez sur l'icône ⚙️ à gauche")
    print("2. Allez dans l'onglet 'Secrets'")
    print("3. Ajoutez un nouveau secret avec:")
    print("   - Nom: OPENAI_API_KEY")
    print("   - Valeur: votre clé API OpenAI")

✅ Clé API configurée avec succès
✅ Test de connexion OpenAI réussi


In [ ]:
# Imports nécessaires
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from chromadb import Settings

try:
    # Configuration des embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # Création de la base vectorielle
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="/content/vector_db_bio",  # Nouveau chemin
        client_settings=Settings(
            is_persistent=True,
            anonymized_telemetry=False
        )
    )

    # Configuration du modèle
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.7
    )

    # Template adapté pour l'article biomédical
    prompt_template = """Analyse le contexte fourni de l'article sur l'IA biomédicale généraliste et réponds à la question.

Contexte: {context}

Question: {question}

Format de réponse:
1. Résumé exécutif (2-3 phrases)
2. Points clés de l'article
3. Implications pratiques
4. Conclusion et perspectives

Réponse:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Configuration QA
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 4}),
        chain_type_kwargs={"prompt": PROMPT}
    )

    # Questions de test
    questions = [
        "Quelles sont les principales innovations en IA biomédicale présentées dans l'article ?",
        "Comment l'article aborde-t-il l'aspect généraliste de l'IA en médecine ?",
        "Quels sont les défis majeurs identifiés dans l'application de l'IA à la biomédecine ?",
        "Décrivez les méthodologies et approches techniques utilisées dans cette recherche."
    ]

    print("\n🔍 Test avec plusieurs questions :")
    for i, question in enumerate(questions, 1):
        print(f"\nQuestion {i}: {question}")
        response = qa_chain.invoke({"query": question})
        print(f"\nRéponse {i}:\n{response['result']}")
        print("-" * 80)

except Exception as e:
    print(f"❌ Erreur: {str(e)}")




🔍 Test avec plusieurs questions :

Question 1: Quelles sont les principales innovations en IA biomédicale présentées dans l'article ?

Réponse 1:
1. Résumé exécutif:
L'article présente Med-PaLM M, le premier système d'IA biomédicale généraliste capable d'interpréter une large gamme de modalités médicales et de performer de manière compétente sur diverses tâches.

2. Points clés de l'article:
- Med-PaLM M est capable d'assimiler et d'encoder des données médicales multimodales à grande échelle et de s'adapter rapidement à de nouveaux contextes cliniques.
- Le système a été évalué par des radiologues et a montré des performances compétentes, voire supérieures aux précédents résultats de référence, sur diverses tâches.
- L'innovation de Med-PaLM M ouvre de nouvelles possibilités d'application allant de la découverte scientifique à la prestation de soins.

3. Implications pratiques:
- Les systèmes d'IA biomédicale généralistes comme Med-PaLM M pourraient améliorer l'accessibilité, l'effica